In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import re
import requests
from datetime import datetime

In [34]:
chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac/chromedriver'
chrome_driver_service = Service(chrome_driver_path)

# Ticket Melon

In [3]:
driver = webdriver.Chrome(service=chrome_driver_service)
driver.maximize_window()
driver.get("https://www.ticketmelon.com")

wait = WebDriverWait(driver, 10) #timeout=10 sec

scroll_distance = 1200
scroll_script = f"window.scrollBy(0, {scroll_distance});"
driver.execute_script(scroll_script)

print("scroll to entertainment :" , scroll_distance, "px")

entertainment_button = wait.until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.slick-slide.slick-active.slick-current')))
entertainment_button.click()

scroll_distance = 1500
scroll_script = f"window.scrollBy(0, {scroll_distance});"
driver.execute_script(scroll_script)

print("scroll to see more :" , scroll_distance, "px")

see_more_button = driver.find_element(
    By.CSS_SELECTOR, '.HomeStyled__EventSeeMore-sc-125xn4a-2.eMhTTB')
driver.execute_script("arguments[0].click();", see_more_button)

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

driver.quit()

scroll to entertainment : 1200 px
scroll to see more : 1500 px


In [4]:
elements = soup.find_all('div',
                         class_='MelonEventCardStyled__EventDetail-sc-8kc3ju-2'
                         )  #event-date, event-name, event-location
event_data = []
for element in elements:
    event_info = {
        "title": element.find('p', class_='event-name').text,
        "date": element.find('p', class_='event-date').text,
        "location": element.find('p', class_='event-location').text
    }
    event_data.append(event_info)
melon_concert = pd.DataFrame(event_data)


## modify date

In [5]:
def extract_dates(date_str):
    date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3})', date_str)
    
    if len(date_parts) == 2:
        start_date, end_date = date_parts
    elif len(date_parts) == 1:
        start_date = end_date = date_parts[0]
    else:
        start_date = end_date = None
    
    return start_date.strip(), end_date.strip() 
melon_concert[['startdate', 'enddate']] = melon_concert['date'].apply(extract_dates).apply(pd.Series)

def calculate_year(date_str):
    # Convert month abbreviations to numeric values
    month_values = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    
    # Extract the month from the date
    month = date_str.split()[1]
    
    if month_values.get(month):
        if month_values[month] >= 8:
            return 2023
        else:
            return 2024
    else:
        return None  # Handle invalid month

melon_concert['startyear'] = melon_concert['startdate'].apply(calculate_year)
melon_concert['endyear'] = melon_concert['enddate'].apply(calculate_year)
melon_concert['startyear'] = melon_concert['startyear'].astype(str)
melon_concert['endyear'] = melon_concert['endyear'].astype(str)

melon_concert['startdate'] = melon_concert['startdate'] + ' ' + melon_concert['startyear']
melon_concert['startdate'] = pd.to_datetime(melon_concert['startdate'], format='%d %b %Y', errors='coerce')
melon_concert['enddate'] = melon_concert['enddate'] + ' ' + melon_concert['endyear']
melon_concert['enddate'] = pd.to_datetime(melon_concert['enddate'], format='%d %b %Y', errors='coerce')

melon_concert.drop(columns=['startyear', 'endyear'], inplace=True)
melon_concert = melon_concert.sort_values(by=['startdate', 'enddate'])

melon_concert.drop(columns=['date'], inplace=True)

In [5]:
# def extract_dates(date_str):
#     date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3})', date_str)
    
#     if len(date_parts) == 2:
#         start_date, end_date = date_parts
#     elif len(date_parts) == 1:
#         start_date = end_date = date_parts[0]
#     else:
#         start_date = end_date = None
    
#     return start_date.strip(), end_date.strip() 
# melon_concert[['startdate', 'enddate']] = melon_concert['date'].apply(extract_dates).apply(pd.Series)

# display(melon_concert)

In [6]:
# melon_concert.info()

In [7]:
# def calculate_year(date_str):
#     # Convert month abbreviations to numeric values
#     month_values = {
#         'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
#         'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
#         'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
#     }
    
#     # Extract the month from the date
#     month = date_str.split()[1]
    
#     if month_values.get(month):
#         if month_values[month] >= 6:
#             return 2023
#         else:
#             return 2024
#     else:
#         return None  # Handle invalid month

# melon_concert['year'] = melon_concert['startdate'].apply(calculate_year)

# melon_concert.head(3)

In [8]:
# melon_concert['year'] = melon_concert['year'].astype(str)
# melon_concert['startdate'] = melon_concert['startdate'] + ' ' + melon_concert['year']
# melon_concert['startdate'] = pd.to_datetime(melon_concert['startdate'], format='%d %b %Y', errors='coerce')

# melon_concert['enddate'] = melon_concert['enddate'] + ' ' + melon_concert['year']
# melon_concert['enddate'] = pd.to_datetime(melon_concert['enddate'], format='%d %b %Y', errors='coerce')

In [6]:
print(melon_concert.shape)
melon_concert.head()


(147, 4)


,title,location,startdate,enddate
145,Swing Era,"Bangkok, Thammasat Association",2023-08-21,2023-08-25
146,Rolling Loud Thailand 2024,"Legend Siam, Pattaya",2023-11-22,2023-11-24
5,เฟี้ยวติวัล 2024,10 จังหวัด 10 สถานที่,2024-01-20,2024-05-19
6,Bangkok Dangerous!,The Comedy Joint Khaosan,2024-01-25,2024-03-14
7,Local Legends,The Comedy Joint Khaosan Bangkok,2024-01-26,2024-03-15


## liite touch.

In [10]:
# melon_concert.drop(columns=['year'], inplace=True)
# melon_concert = melon_concert.sort_values(by=['startdate', 'enddate'])

In [11]:
# melon_concert.drop(columns=['date'], inplace=True)

In [7]:
display(melon_concert)

,title,location,startdate,enddate
145,Swing Era,"Bangkok, Thammasat Association",2023-08-21,2023-08-25
146,Rolling Loud Thailand 2024,"Legend Siam, Pattaya",2023-11-22,2023-11-24
5,เฟี้ยวติวัล 2024,10 จังหวัด 10 สถานที่,2024-01-20,2024-05-19
6,Bangkok Dangerous!,The Comedy Joint Khaosan,2024-01-25,2024-03-14
7,Local Legends,The Comedy Joint Khaosan Bangkok,2024-01-26,2024-03-15
...,...,...,...,...
140,CosYume Wedding Gala,St. Tropez,2024-06-29,2024-06-29
141,UnKonscious Croatia 2024,Zrce beach,2024-07-16,2024-07-20
142,"""เตรียมพร้อมกับก้าวใหม่ในต่างแดน"" 18/07 9.00-...",Bangkok,2024-07-18,2024-07-18
143,WHITE PARTY VIETNAM 2024,"Phu Tho Indoor Stadium, Ho Chi Minh City",2024-07-20,2024-07-21


In [8]:

melon_concert['vendor'] = 'Ticketmelon'

In [9]:
nan_rows = melon_concert.loc[melon_concert.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [title, location, startdate, enddate, vendor]
Index: []


In [10]:
# oct_con = melon_concert[melon_concert['startdate'].dt.month == 10]
# display(oct_con)

# TTM

## make df

In [11]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}

TTMbigdata = requests.get("https://www.thaiticketmajor.com/concert/", headers=headers)


In [12]:
# Check if the request was successful (status code 200 indicates success)
if TTMbigdata.status_code == 200:
    # Print the content of the response (HTML content in this case)
    print(TTMbigdata.text)
else:
    # Print an error message if the request was not successful
    print("Failed to retrieve data. Status code:", TTMbigdata.status_code)


<!DOCTYPE html>
<html lang="en" class="TH">
<head>
	<meta charset="utf-8"/>
	<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/>
	<!-- <meta name="apple-mobile-web-app-capable" content="yes" /> -->
	<meta name="apple-mobile-web-app-title" content="THAITICKETMAJOR"/>
	<meta name="viewport" content="width=device-width, initial-scale=1.0" />
	<title>คอนเสิร์ต จองบัตร ซื้อบัตร คอนเสิร์ต ทั่วประเทศไทย - ThaiTicketMajor.com : Thailand Tickets Center</title>
	<meta name="description" content="Thailand Tickets Center : แหล่งรวม บัตร คอนเสิร์ต ทั่วไทย ใหญ่ที่สุด มั่นใจกับระบบขายบัตร จองบัตร คอนเสิร์ต อัจฉริยะ หลากหลายช่องทาง การให้บริการ  - ไทยทิคเก็ตเมเจอร์ จำกัด เป็นตัวแทนจำหน่ายบัตรแบบครบวงจรเต็มรูปแบบแห่งแรกในประเทศไทย และ มุ่งสร้างสรรค์บริการรูปแบบใหม่ รวมทั้งเป็นตัวแทนจำหน่ายบัตรเข้าชมการแสดงแบบ Online Realtime ในทุกๆ ช่องทางการจัดจำหน่าย"/>
	<meta name="keywords" content="คอนเสิร์ต, ซื้อบัตร คอนเสิร์ต, จองบัตร คอนเสิร์ต, concert, concerts, live concert, concert rock, concert 

In [13]:
# TTMsoup.find_all("div", class_="box-txt")


NameError: name 'TTMsoup' is not defined

In [15]:
TTMsoup = BeautifulSoup(TTMbigdata.text, "html.parser")
TTMsoup.find_all("div", class_="box-txt")

div_elements = TTMsoup.find_all("div", class_="box-txt")
event_data = []

for div in div_elements:
    event_info = {
        "title": div.find("a", class_="title").text if div.find("a", class_="title") else "",
        "date": div.find("span", class_="datetime").text if div.find("span", class_="datetime") else "",
        "location": div.find("a", class_="venue").text if div.find("a", class_="venue") else ""
    }
    event_data.append(event_info)


ttm_concert = pd.DataFrame(event_data)
# ttm_concert = ttm_concert[~ttm_concert['date'].str.contains('Public Sale')]
ttm_concert.head(5)

,title,date,location
0,LANY - a beautiful blur: the world tour 2024,6 ต.ค. 2567,ชาเลนเจอร์ฮอลล์ อิมแพ็ค เมืองทองธานี
1,The Sign 1st Fan Meeting : Lost in the jungle,6 เม.ย. 2567,เอ็มซีซี ฮอลล์ เดอะมอลล์ไลฟ์สโตร์ บางกะปิ
2,Polca Time Traveling Concert,วันเสาร์ที่ 27 เมษายน 2567,ยูเนี่ยน ฮอลล์ ชั้น F6 ศูนย์การค้ายูเนี่ยน มอลล์
3,BABII 24/7 CONCERT,วันเสาร์ที่ 20 เมษายน 2567,ยูเนี่ยน ฮอลล์ ชั้น F6 ศูนย์การค้ายูเนี่ยน มอลล์
4,Vir Das : Mind Fool Tour in Bangkok,19 พ.ค. 2567,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน


## modify date

In [16]:
thai_to_english_month = {
    "ม.ค.": "Jan",
    "ก.พ.": "Feb",
    "มี.ค.": "Mar",
    "เม.ย.": "Apr",
    "พ.ค.": "May",
    "มิ.ย.": "Jun",
    "ก.ค.": "Jul",
    "ส.ค.": "Aug",
    "ก.ย.": "Sep",
    "ต.ค.": "Oct",
    "พ.ย.": "Nov",
    "ธ.ค.": "Dec"
}

def replace_thai_month_with_english(date_str):
    for thai_month, english_month in thai_to_english_month.items():
        date_str = date_str.replace(thai_month, english_month)
    return date_str

# Apply the replacement function to the 'date' column
ttm_concert['date'] = ttm_concert['date'].apply(replace_thai_month_with_english)
ttm_concert.head(1)

,title,date,location
0,LANY - a beautiful blur: the world tour 2024,6 Oct 2567,ชาเลนเจอร์ฮอลล์ อิมแพ็ค เมืองทองธานี


In [17]:
ttm_concert.reset_index(drop=True, inplace=True)
display(ttm_concert)

,title,date,location
0,LANY - a beautiful blur: the world tour 2024,6 Oct 2567,ชาเลนเจอร์ฮอลล์ อิมแพ็ค เมืองทองธานี
1,The Sign 1st Fan Meeting : Lost in the jungle,6 Apr 2567,เอ็มซีซี ฮอลล์ เดอะมอลล์ไลฟ์สโตร์ บางกะปิ
2,Polca Time Traveling Concert,วันเสาร์ที่ 27 เมษายน 2567,ยูเนี่ยน ฮอลล์ ชั้น F6 ศูนย์การค้ายูเนี่ยน มอลล์
3,BABII 24/7 CONCERT,วันเสาร์ที่ 20 เมษายน 2567,ยูเนี่ยน ฮอลล์ ชั้น F6 ศูนย์การค้ายูเนี่ยน มอลล์
4,Vir Das : Mind Fool Tour in Bangkok,19 May 2567,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน
5,Arch Enemy Live In BangkokDeceivers Asia Tour ...,21 May 2567,Mr.FOX Live House
6,Thailand Phil 2023/24 Season,2 Dec 2566 - 7 Sep 2567,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา
7,CHA EUN-WOO 2024 Just One 10 Minute [Mystery E...,9 Mar 2567,"Exhibition Hall 3-4, ศูนย์ประชุมแห่งชาติสิริกิติ์"
8,RBSO 2024 : Celebrating International Women's Day,9 Mar 2567,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่
9,Music @ Mahidol,12 Mar 2567,หอแสดงดนตรี วิทยาลัยดุริยางคศิลป์ ม.มหิดล(ศาลายา)


In [18]:
#remove non-numeric date's starting character
ttm_concert = ttm_concert[ttm_concert['date'].apply(lambda x: x[0].isdigit())]

In [19]:
# # ttm_concert.loc[1,['date']] = '23 Dec 2566'
# rows = [7, 23]
# ttm_concert.drop(rows, inplace=True)

In [20]:
ttm_concert.shape

(33, 3)

In [21]:
# Function to subtract 543 from the year part of a date string
def subtract_543_from_year(date_str):
    # Split the date string by spaces
    parts = date_str.split()
    
    # Check if the date string has the format "DD MMM YYYY - DD MMM YYYY"
    if len(parts) == 7:
        # Extract the year parts
        year_parts = parts[2], parts[-1]
        
        # Subtract 543 from each year part
        updated_year_parts = [str(int(year) - 543) for year in year_parts]
        
        # Replace the year parts in the date string
        parts[2], parts[-1] = updated_year_parts
        
        # Join the updated parts to form the new date string
        updated_date_str = " ".join(parts)
        return updated_date_str
    else:
        # Extract the year part
        year_part = parts[-1]
        
        # Subtract 543 from the year part
        updated_year_part = str(int(year_part) - 543)
        
        # Replace the year part in the date string
        parts[-1] = updated_year_part
        
        # Join the updated parts to form the new date string
        updated_date_str = " ".join(parts)
        return updated_date_str

# Apply the subtraction function to the 'date' column
ttm_concert['date'] = ttm_concert['date'].apply(subtract_543_from_year)

ttm_concert.head(3)


,title,date,location
0,LANY - a beautiful blur: the world tour 2024,6 Oct 2024,ชาเลนเจอร์ฮอลล์ อิมแพ็ค เมืองทองธานี
1,The Sign 1st Fan Meeting : Lost in the jungle,6 Apr 2024,เอ็มซีซี ฮอลล์ เดอะมอลล์ไลฟ์สโตร์ บางกะปิ
4,Vir Das : Mind Fool Tour in Bangkok,19 May 2024,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน


In [22]:
display(ttm_concert)

,title,date,location
0,LANY - a beautiful blur: the world tour 2024,6 Oct 2024,ชาเลนเจอร์ฮอลล์ อิมแพ็ค เมืองทองธานี
1,The Sign 1st Fan Meeting : Lost in the jungle,6 Apr 2024,เอ็มซีซี ฮอลล์ เดอะมอลล์ไลฟ์สโตร์ บางกะปิ
4,Vir Das : Mind Fool Tour in Bangkok,19 May 2024,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน
5,Arch Enemy Live In BangkokDeceivers Asia Tour ...,21 May 2024,Mr.FOX Live House
6,Thailand Phil 2023/24 Season,2 Dec 2023 - 7 Sep 2024,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา
7,CHA EUN-WOO 2024 Just One 10 Minute [Mystery E...,9 Mar 2024,"Exhibition Hall 3-4, ศูนย์ประชุมแห่งชาติสิริกิติ์"
8,RBSO 2024 : Celebrating International Women's Day,9 Mar 2024,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่
9,Music @ Mahidol,12 Mar 2024,หอแสดงดนตรี วิทยาลัยดุริยางคศิลป์ ม.มหิดล(ศาลายา)
10,Tom Jones : Ages & Stage Tour - Bangkok,14 Mar 2024,ยูโอบี ไลฟ์
11,The Golden Song คอนเสิร์ตเพลงเพราะ,16 Mar 2024 - 17 Mar 2024,เมืองไทยรัชดาลัย เธียเตอร์


### clean

In [23]:
# ttm_concert = ttm_concert.drop(28)

In [ ]:
# display(ttm_concert)

In [23]:
def extract_dates(date_str):
    date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3}\s*\d{4})', date_str)
    
    if len(date_parts) == 2:
        start_date, end_date = date_parts
    elif len(date_parts) == 1:
        start_date = end_date = date_parts[0]
    else:
        start_date = end_date = None
    
    return start_date.strip(), end_date.strip()

ttm_concert[['startdate', 'enddate']] =ttm_concert['date'].apply(extract_dates).apply(pd.Series)
ttm_concert.head(3)

,title,date,location,startdate,enddate
0,LANY - a beautiful blur: the world tour 2024,6 Oct 2024,ชาเลนเจอร์ฮอลล์ อิมแพ็ค เมืองทองธานี,6 Oct 2024,6 Oct 2024
1,The Sign 1st Fan Meeting : Lost in the jungle,6 Apr 2024,เอ็มซีซี ฮอลล์ เดอะมอลล์ไลฟ์สโตร์ บางกะปิ,6 Apr 2024,6 Apr 2024
4,Vir Das : Mind Fool Tour in Bangkok,19 May 2024,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน,19 May 2024,19 May 2024


In [24]:
def convert_date_format(date_str):
    # Parse the date string into a datetime object
    date_obj = datetime.strptime(date_str, '%d %b %Y')
    
    # Format the datetime object as 'YYYY-MM-DD'
    formatted_date = date_obj.strftime('%Y-%m-%d')
    
    return formatted_date

ttm_concert['startdate'] = ttm_concert['startdate'].apply(convert_date_format)
ttm_concert['enddate'] = ttm_concert['enddate'].apply(convert_date_format)

ttm_concert.head()

,title,date,location,startdate,enddate
0,LANY - a beautiful blur: the world tour 2024,6 Oct 2024,ชาเลนเจอร์ฮอลล์ อิมแพ็ค เมืองทองธานี,2024-10-06,2024-10-06
1,The Sign 1st Fan Meeting : Lost in the jungle,6 Apr 2024,เอ็มซีซี ฮอลล์ เดอะมอลล์ไลฟ์สโตร์ บางกะปิ,2024-04-06,2024-04-06
4,Vir Das : Mind Fool Tour in Bangkok,19 May 2024,โรงละครเคแบงก์สยามพิฆเนศ ชั้น7 สยามสแควร์วัน,2024-05-19,2024-05-19
5,Arch Enemy Live In BangkokDeceivers Asia Tour ...,21 May 2024,Mr.FOX Live House,2024-05-21,2024-05-21
6,Thailand Phil 2023/24 Season,2 Dec 2023 - 7 Sep 2024,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา,2023-12-02,2024-09-07


In [25]:
# ttm_concert.astype(da)

In [26]:
ttm_concert['startdate'] = pd.to_datetime(ttm_concert['startdate'])
ttm_concert['enddate'] = pd.to_datetime(ttm_concert['enddate'])
ttm_concert = ttm_concert.sort_values(['startdate', 'enddate'])
ttm_concert = ttm_concert.drop(columns=['date'], axis=1)
display(ttm_concert)

,title,location,startdate,enddate
6,Thailand Phil 2023/24 Season,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา,2023-12-02,2024-09-07
7,CHA EUN-WOO 2024 Just One 10 Minute [Mystery E...,"Exhibition Hall 3-4, ศูนย์ประชุมแห่งชาติสิริกิติ์",2024-03-09,2024-03-09
8,RBSO 2024 : Celebrating International Women's Day,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่,2024-03-09,2024-03-09
9,Music @ Mahidol,หอแสดงดนตรี วิทยาลัยดุริยางคศิลป์ ม.มหิดล(ศาลายา),2024-03-12,2024-03-12
10,Tom Jones : Ages & Stage Tour - Bangkok,ยูโอบี ไลฟ์,2024-03-14,2024-03-14
12,The Magic Moment Concert ตอน The Power of Love...,"อิมแพ็ค เอ็กซิบิชั่น ฮอลล์ 5, เมืองทองธานี",2024-03-16,2024-03-16
13,ITZY 2nd WORLD TOUR < BORN TO BE > IN BANGKOK,อิมแพ็ค อารีน่า เมืองทองธานี,2024-03-16,2024-03-16
11,The Golden Song คอนเสิร์ตเพลงเพราะ,เมืองไทยรัชดาลัย เธียเตอร์,2024-03-16,2024-03-17
15,KISS OF LIFE 'DEAR KISSY' 1st Fan Meeting in B...,"แจ้งวัฒนะ ฮอลล์, เซ็นทรัลแจ้งวัฒนะ ชั้น 5.5",2024-03-17,2024-03-17
16,TOKYO SYMPHONY ORCHESTRA SPECIAL CONCERT IN BA...,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่,2024-03-22,2024-03-22


In [27]:
ttm_concert['vendor'] = 'TTM'

In [28]:
display(ttm_concert)

,title,location,startdate,enddate,vendor
6,Thailand Phil 2023/24 Season,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา,2023-12-02,2024-09-07,TTM
7,CHA EUN-WOO 2024 Just One 10 Minute [Mystery E...,"Exhibition Hall 3-4, ศูนย์ประชุมแห่งชาติสิริกิติ์",2024-03-09,2024-03-09,TTM
8,RBSO 2024 : Celebrating International Women's Day,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่,2024-03-09,2024-03-09,TTM
9,Music @ Mahidol,หอแสดงดนตรี วิทยาลัยดุริยางคศิลป์ ม.มหิดล(ศาลายา),2024-03-12,2024-03-12,TTM
10,Tom Jones : Ages & Stage Tour - Bangkok,ยูโอบี ไลฟ์,2024-03-14,2024-03-14,TTM
12,The Magic Moment Concert ตอน The Power of Love...,"อิมแพ็ค เอ็กซิบิชั่น ฮอลล์ 5, เมืองทองธานี",2024-03-16,2024-03-16,TTM
13,ITZY 2nd WORLD TOUR < BORN TO BE > IN BANGKOK,อิมแพ็ค อารีน่า เมืองทองธานี,2024-03-16,2024-03-16,TTM
11,The Golden Song คอนเสิร์ตเพลงเพราะ,เมืองไทยรัชดาลัย เธียเตอร์,2024-03-16,2024-03-17,TTM
15,KISS OF LIFE 'DEAR KISSY' 1st Fan Meeting in B...,"แจ้งวัฒนะ ฮอลล์, เซ็นทรัลแจ้งวัฒนะ ชั้น 5.5",2024-03-17,2024-03-17,TTM
16,TOKYO SYMPHONY ORCHESTRA SPECIAL CONCERT IN BA...,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่,2024-03-22,2024-03-22,TTM


# The Concert

In [35]:
driver = webdriver.Chrome(service=chrome_driver_service)
driver.get("https://www.theconcert.com/concert")

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

last_height = 0
print(last_height)
time.sleep(5)
count = 0
while(True) : 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    print(new_height)
    if count > 1 :
        break 
    if new_height == last_height :
        count += 1
    else :
        last_height = new_height
        count = 0

wait = WebDriverWait(driver, 10)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

0
2600
3922
4816
5934
5934
5934
5934


In [36]:
elements = soup.find_all('div',
                         class_='recom-detail-box'
                         )  #conert-recom-title, concert-recom-date, concert-recom-date, concert-recom-local
event_data = []
for element in elements:
    event_info = {
        "title": element.find('div', class_='concert-recom-title').text,
        "date": element.find('div', class_='concert-recom-date').text,
        "location": element.find('div', class_='concert-recom-local').text
    }
    event_data.append(event_info)

the_concert = pd.DataFrame(event_data)
the_concert.head(3)

,title,date,location
0,GAVIN:D x LAMAYA,8 มี.ค. 24,Lamaya
1,Liquid Room Present Greg Downey,8 มี.ค. 24,Liquid Room
2,EPIC Pool Party at Pullman Bangkok King Power,9 มี.ค. 24,Pullman Bangkok King Power


In [37]:
the_concert.tail()

,title,date,location
51,RimLay Beach Life Festival,6 - 7 ก.ค. 24,"SAMILA BEACH, SONGKHLA"
52,Your Folk Fes #4 2024,3 - 4 ส.ค. 24,บ้านนอกคอกควายค๊อฟฟี่
53,LONGLAY BEACH LIFE FESTIVAL 2024,7 - 8 ธ.ค. 24,Diamond Beach (Longlay Beach)
54,ไร่อมรแคมป์ปิ้งเฟสติวัลครั้งที่4,14 - 15 ธ.ค. 24,ไร่อมรแคมป์ปิ้งเฟสติวัล
55,Bank Preeti Live in 91bar,18 - 20 มี.ค. 24,91 Bar & Restaurants


## mo date

In [38]:
thai_to_english_month = {
    "ม.ค.": "Jan",
    "ก.พ.": "Feb",
    "มี.ค.": "Mar",
    "เม.ย.": "Apr",
    "พ.ค.": "May",
    "มิ.ย.": "Jun",
    "ก.ค.": "Jul",
    "ส.ค.": "Aug",
    "ก.ย.": "Sep",
    "ต.ค.": "Oct",
    "พ.ย.": "Nov",
    "ธ.ค.": "Dec"
}

def replace_thai_month_with_english(date_str):
    for thai_month, english_month in thai_to_english_month.items():
        date_str = date_str.replace(thai_month, english_month)
    return date_str

the_concert['date'] = the_concert['date'].apply(replace_thai_month_with_english)
display(the_concert)


,title,date,location
0,GAVIN:D x LAMAYA,8 Mar 24,Lamaya
1,Liquid Room Present Greg Downey,8 Mar 24,Liquid Room
2,EPIC Pool Party at Pullman Bangkok King Power,9 Mar 24,Pullman Bangkok King Power
3,FALLEN for SEO IN GUK 1st FAN MEETING IN BANGKOK,9 Mar 24,"Ultra Arena, Bravo BKK"
4,เต้นรำครั้งสุดท้าย THE YERS FAREWELL CONCERT ท...,9 Mar 24,Thunder Dome
5,Ink Waruntorn @HOUSE Klong 6,12 Mar 24,HOUSE Klong 6 เฮ้าส์ คลองหก
6,Jay Jetrin x Lamaya,15 Mar 24,Lamaya
7,รถตู้รับส่ง @Brovo 15 Mar 2024,15 Mar 24,BRAVO BKK
8,HYBS : WELL DONE CONCERT,15 Mar 24,MGI HALL ชั้น 6 ศูนย์การค้า SHOW DC
9,รถตู้รับส่ง @Brovo 16 Mar 2024,16 Mar 24,BRAVO BKK


In [39]:
def change_year(date_str):
    # Use regular expression to find and replace the year
    updated_date_str = re.sub(r'\b23$', '2023', date_str)
    updated_date_str = re.sub(r'\b24$', '2024', updated_date_str)
    return updated_date_str

the_concert['date'] = the_concert['date'].apply(change_year)
the_concert.head(1)

,title,date,location
0,GAVIN:D x LAMAYA,8 Mar 2024,Lamaya


In [40]:
def split_date(date_str):
    parts = date_str.split('-')
    start_date = parts[0].strip()
    
    if len(parts) > 1:
        end_date = parts[1].strip()
        end_day, end_month, year = end_date.split()
        try:
            start_day, start_month = start_date.split()
            print(start_day, start_month)

        except ValueError:
            start_day = start_date
            start_month = end_month  # or any other suitable default value

        start_date = f"{start_day} {start_month} {year}"
    else:
        end_date = start_date
        
    return start_date, end_date
# Apply the function to the 'date' column
the_concert[['startdate', 'enddate']] = the_concert['date'].apply(split_date).apply(pd.Series)
the_concert.head(3)

,title,date,location,startdate,enddate
0,GAVIN:D x LAMAYA,8 Mar 2024,Lamaya,8 Mar 2024,8 Mar 2024
1,Liquid Room Present Greg Downey,8 Mar 2024,Liquid Room,8 Mar 2024,8 Mar 2024
2,EPIC Pool Party at Pullman Bangkok King Power,9 Mar 2024,Pullman Bangkok King Power,9 Mar 2024,9 Mar 2024


## little touch

In [41]:
the_concert = the_concert.drop(columns=['date'], axis=1)
the_concert['startdate'] = pd.to_datetime(the_concert['startdate'], format='%d %b %Y', errors='coerce')
the_concert['enddate'] = pd.to_datetime(the_concert['enddate'], format='%d %b %Y', errors='coerce')

the_concert = the_concert.sort_values(['startdate', 'enddate'])
the_concert.reset_index(drop=True, inplace=True)
display(the_concert)

,title,location,startdate,enddate
0,GAVIN:D x LAMAYA,Lamaya,2024-03-08,2024-03-08
1,Liquid Room Present Greg Downey,Liquid Room,2024-03-08,2024-03-08
2,EPIC Pool Party at Pullman Bangkok King Power,Pullman Bangkok King Power,2024-03-09,2024-03-09
3,FALLEN for SEO IN GUK 1st FAN MEETING IN BANGKOK,"Ultra Arena, Bravo BKK",2024-03-09,2024-03-09
4,เต้นรำครั้งสุดท้าย THE YERS FAREWELL CONCERT ท...,Thunder Dome,2024-03-09,2024-03-09
5,Ink Waruntorn @HOUSE Klong 6,HOUSE Klong 6 เฮ้าส์ คลองหก,2024-03-12,2024-03-12
6,Jay Jetrin x Lamaya,Lamaya,2024-03-15,2024-03-15
7,รถตู้รับส่ง @Brovo 15 Mar 2024,BRAVO BKK,2024-03-15,2024-03-15
8,HYBS : WELL DONE CONCERT,MGI HALL ชั้น 6 ศูนย์การค้า SHOW DC,2024-03-15,2024-03-15
9,รถตู้รับส่ง @Brovo 16 Mar 2024,BRAVO BKK,2024-03-16,2024-03-16


In [42]:
the_concert['vendor'] = 'The Concert'

In [43]:
nan_rows = the_concert.loc[the_concert.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [title, location, startdate, enddate, vendor]
Index: []


# Combined 

In [44]:
# df = pd.concat([melon_concert, ttm_concert, the_concert])
df = pd.concat([melon_concert, the_concert])
df = df.sort_values(['startdate', 'enddate'])
df.reset_index(drop=True, inplace=True)
display(df)

,title,location,startdate,enddate,vendor
0,Swing Era,"Bangkok, Thammasat Association",2023-08-21,2023-08-25,Ticketmelon
1,Rolling Loud Thailand 2024,"Legend Siam, Pattaya",2023-11-22,2023-11-24,Ticketmelon
2,เฟี้ยวติวัล 2024,10 จังหวัด 10 สถานที่,2024-01-20,2024-05-19,Ticketmelon
3,Bangkok Dangerous!,The Comedy Joint Khaosan,2024-01-25,2024-03-14,Ticketmelon
4,Local Legends,The Comedy Joint Khaosan Bangkok,2024-01-26,2024-03-15,Ticketmelon
...,...,...,...,...,...
198,WHITE PARTY VIETNAM 2024,"Phu Tho Indoor Stadium, Ho Chi Minh City",2024-07-20,2024-07-21,Ticketmelon
199,Anime Event Thailand #2,Fashion Island,2024-07-27,2024-07-28,Ticketmelon
200,Your Folk Fes #4 2024,บ้านนอกคอกควายค๊อฟฟี่,2024-08-03,2024-08-04,The Concert
201,LONGLAY BEACH LIFE FESTIVAL 2024,Diamond Beach (Longlay Beach),2024-12-07,2024-12-08,The Concert


In [45]:
import datetime

path = '/Users/got/Documents/retune/t+im/concert_data/concert_data'
today_date = datetime.datetime.now().strftime('%d-%m-%Y')
path = path + '_' + today_date + '.csv'
print(path)
df.to_csv(path, index=False)

/Users/got/Documents/retune/t+im/concert_data/concert_data_08-03-2024.csv
